# Curso Teste A/B/n - PA James

## 0.1. Imports

In [1]:
import pandas as pd

# 1.0. Carregar Dados

# 2.0. Design de Experimentos

## 2.1. Formulação das Hipóteses

## 2.2. parâmetros do Experimento

# 3.0. Conversão da página em Faturamento